# imports:

In [1]:
from os import listdir,mkdir,makedirs
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr,fisher_exact
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics.SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from skbio.stats.distance import mantel
from scipy.spatial.distance import braycurtis, pdist, euclidean


from GeneralFeaturePhenotypeInteractions.Feature_phenotype_functions import * 
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions import *
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions2 import *
from SampleLists.SampleFileFunctions import *
from PhenotypicData.PhenotypeGenerationFunctions import *
from CardioProject.CardioFunctions import *
from PredictionPipeline.PredictionFunctions import *
from TCR_feature_generation.SubsamplingFunctions import *

import os
from Utils import cacheOnDisk
from SegalQueue.qp import qp,fakeqp
from addloglevels import sethandlers

#ML imports:
from xgboost import XGBClassifier
import lightgbm as lgb
from collections import OrderedDict
from sklearn.model_selection import GroupKFold,StratifiedKFold, KFold
import statsmodels.formula.api as sm
from sklearn.linear_model import LogisticRegression
import shap
from sklearn import metrics, preprocessing
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel,SelectKBest,chi2,mutual_info_classif,f_classif
from sklearn.naive_bayes import GaussianNB

MyPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF'


done1
stop
stop
done1
stop
done
done
done
done
stop
stop
end of function!!!


/usr/wisdom/python/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


end of function!!!


In [2]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate

'12102018'

# get DFs:

In [3]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP530','rb') as fp:
    PNP530=pickle.load(fp)
sampleList=PNP530

In [12]:
#get PNP phenotype df:
datasetFolder='%s/TCR_real_data' %MyPath
f2='%s/NewPhenotypicData/newPhenotypesPNPAllInfo_withDummies.xlsx' %datasetFolder
phenotypeDF=pd.read_excel(f2).set_index('BD')

phenotypeDF=phenotypeDF.loc[PNP530,:]
print phenotypeDF.shape
print phenotypeDF.iloc[:4,:4]


(530, 21)
        Age  Gender        BMI  Creatinine
BD                                        
BD406  65.0  Female  27.181630       1.070
BD228  32.0    Male  28.727377       1.080
BD714  39.0    Male  27.216277       1.015
BD808  26.0  Female  18.423259       0.910


In [13]:
#get TCRfeature df:
f3='%s/TCR_real_data/Predictions/featureDFs/allTCRfeatures_percShared10_.dat' %MyPath
allFeatures_PNP530=pd.read_pickle(f3).loc[PNP530,:]
print allFeatures_PNP530.shape
print allFeatures_PNP530.iloc[:4,:4]

(530, 671)
       AA count_0  AA count_1  AA count_T     D01_0
BD406       190.0      8017.0      8207.0  0.381305
BD228       357.0     12365.0     12722.0  0.383422
BD714       208.0     10560.0     10768.0  0.459386
BD808       359.0     19513.0     19872.0  0.508576


In [9]:
num_phens=['Age','BMI','Hemoglobin','WBC','CRP_wr','AST', 'HDL', 'Total Cholesterol','eGFR by CKD-EPI',
      'HbA1C', 'Glucose','WakeupGlucose']
cat_phens=['Smoking Status_Past','Smoking Status_Yes','Gender_Male']

featurelist=['normSeqNums_per2000_NT_T','top10clonal_nt_T','top1000clonal_nt_T','frequencyCount (%)_mean_T',
              'frequencyCount (%)_max_T','frequencyCount (%)_std_T','shannon_nt_T','cdr3Length_mean_T',
              'n1Insertion_mean_T','n2Insertion_mean_T','mean_nt_per_aa_T','NT count_T','gc_content_T','PC1','PC2']


# calculate association with binary variables:

## using ROC_AUC

In [ ]:
feature_auc_df = pd.DataFrame()
annDF2=allFeatures_PNP530[featurelist].copy()
phenotypeDF2 = phenotypeDF[cat_phens].copy().loc[annDF2.index,:]

print phenotypeDF2.head()

print annDF2.index[:10]
print phenotypeDF2.index[:10]
print annDF2.shape
print phenotypeDF2.shape

for n,feature in enumerate(annDF2.columns):
    annDF2[feature]=annDF2[feature].fillna(0)
    for k,phen in enumerate(phenotypeDF2.columns):
        df=pd.DataFrame()
        phenotypeDF2[phen]=phenotypeDF2[phen].fillna(phenotypeDF2[phen].median())    
        print n,feature,k,phen

        fpr, tpr, thresholds = metrics.roc_curve(phenotypeDF2[phen], annDF2[feature])
        roc_auc = metrics.auc(fpr, tpr)
        df.loc[0,'Feature']=feature
        df.loc[0,'Phenotype']=phen
        df.loc[0,'ROC-AUC']=max(roc_auc, 1-roc_auc)

        feature_auc_df=pd.concat([feature_auc_df,df])


In [ ]:
feature_auc_df=feature_auc_df.sort_values(by='ROC-AUC',ascending=False)
feature_auc_df.head()

In [ ]:
f7='%s/TCR_real_data/TCR_phenotype_relations/feature_phen_auc_df.xlsx' %MyPath
feature_auc_df.to_excel(f7)

## using t-test

the function calc_catPhens_TCRfeatures_associations_ttest was copied to feature_phenotype_functions.py

In [17]:
featureDF=allFeatures_PNP530[featurelist].copy()
featureDFname='mostInteresting'
phenDF = phenotypeDF[cat_phens].copy()
phenDFname='catPhens'
FDR=0.1

feature_phen_ttest=calc_catPhens_TCRfeatures_associations_ttest(featureDF,featureDFname,phenDF,phenDFname,FDR)

f1='%s/TCR_real_data/TCR_phenotype_relations/%sTCRfeatures_%s_ttest.xlsx' %(MyPath,featureDFname,phenDFname)
feature_phen_ttest.to_excel(f1)

('featureDF shape is', (530, 15))
('fisrt samples in featureDF are:', Index([u'BD406', u'BD228', u'BD714', u'BD808'], dtype='object'))
('phenDF shape is', (530, 3))
('fisrt samples in phenDF are:', Index([u'BD406', u'BD228', u'BD714', u'BD808'], dtype='object'))


# calculate association with continous variables:

In [ ]:
the function calc_numPhens_TCRfeatures_associations_correlation was copied to feature_phenotype_functions.py

In [25]:
featureDF=allFeatures_PNP530[featurelist].copy()
featureDFname='mostInteresting'
phenDF = phenotypeDF[num_phens].copy()
phenDFname='num_phens'
testType='pearson'
FDR=0.1


feature_phen_pearson=calc_numPhens_TCRfeatures_associations_correlation(featureDF,featureDFname,phenDF,phenDFname,testType,FDR=0.1)

f2='%s/TCR_real_data/TCR_phenotype_relations/%sTCRfeatures_%s_%sTest.xlsx' %(MyPath,featureDFname,phenDFname,testType)
feature_phen_pearson.to_excel(f2)

('featureDF shape is', (530, 15))
('fisrt samples in featureDF are:', Index([u'BD406', u'BD228', u'BD714', u'BD808'], dtype='object'))
('phenDF shape is', (530, 12))
('fisrt samples in phenDF are:', Index([u'BD406', u'BD228', u'BD714', u'BD808'], dtype='object'))


In [26]:
feature_phen_pearson.head()

,Feature,Phenotype,r,p,testType,Sig by bonferroni corrected pVal,sig. by FDR=0.1
0,PC2,Age,-0.347063,2.309359e-16,pearson,1,1
0,top1000clonal_nt_T,Age,0.324330,2.255600e-14,pearson,1,1
0,shannon_nt_T,Age,-0.321041,4.242588e-14,pearson,1,1
0,PC1,Age,-0.297256,3.265018e-12,pearson,1,1
0,top10clonal_nt_T,Age,0.289176,1.307272e-11,pearson,1,1
